# R script for building GRNs

Requires ChIP-seq (of gene of interest) peak calls annotated to the nearest promoter (Or alternative peak-promoter assignment). This script is built on output from Homer annotatePeaks.pl.

Requires differential RNA-seq data, after knockdown of gene of interest. This script is built on output from EdgeR.


## load packages

In [4]:
library(tidyverse)
library(annotables)
library(GeneNetworkBuilder)


## Convert ChIP-seq peak file into edge list

In [8]:
suppressMessages(read_tsv("../data/ChIPseq_tables/MLL-AF4.ann.bed")) %>%
    head(2)

PeakID (cmd=annotatePeaks.pl /t1-data/user/rthorne/Analysis-Scripts/SMYD2/data/MLL_AF4_Peaks_hg19.bed hg19 -noann),Chr,Start,End,Strand,Peak Score,Focus Ratio/Region Size,Annotation,Detailed Annotation,Distance to TSS,Nearest PromoterID,Entrez ID,Nearest Unigene,Nearest Refseq,Nearest Ensembl,Gene Name,Gene Alias,Gene Description,Gene Type
<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<lgl>,<lgl>,<lgl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
PTPN1-2,chr20,49134420,49136313,+,0,NA,NA,NA,8508,NM_001278618,5770,Hs.417549,NM_002827,HPRD:01477,PTPN1,PTP1B,"protein tyrosine phosphatase, non-receptor type 1",protein-coding
SENP6,chr6,76308942,76340220,+,0,NA,NA,NA,12959,NM_001100409,26054,Hs.485784,NM_015571,HPRD:05417,SENP6,SSP1|SUSP1,SUMO1/sentrin specific peptidase 6,protein-coding


In [9]:
binding.table <- suppressMessages(read_tsv("../data/ChIPseq_tables/MLL-AF4.ann.bed")) %>%
    mutate(
        from = "4297", # Entrez ID for KMT2A (gene of interest). Replace as required.
        to = as.character(`Entrez ID`)
            ) %>%
    select(from, to) %>%
    unique() %>%
    as.data.frame()

head(binding.table)

,from,to
,<chr>,<chr>
1,4297,5770
2,4297,26054
3,4297,1997
4,4297,4134
5,4297,3185
6,4297,2770


## Load underlying network

In [10]:
data("hs.interactionmap") # Underlying network, sourced from GeneNetworkBuilder

In [11]:
head(hs.interactionmap)

,from,to
,<fct>,<fct>
1,1874,574512
2,4808,391059
3,2002,391059
4,7050,391059
5,4087,391059
6,6775,391059


## Import RNA-seq data and convert gene symbols into Entrez IDs

In [13]:
deg.filePath <- "../data/RNAseq_tables/MLLAF4_KD/contrast_SIMM_vs_SIMA6_counts.tsv"

de.genes <- suppressMessages(read_tsv(deg.filePath)) %>%
    select(Geneid, logFC, FDR)

head(de.genes, 3)

Geneid,logFC,FDR
<chr>,<dbl>,<dbl>
PROM1,-1.918198,3.606216e-275
ANXA2R,-2.123526,3.086909e-222
LOC101927497,-2.178858,3.824309e-168


Prepare gene table for lookup joins, using annotables.

In [15]:
# Prepare gene table for lookup joins
lookup <- grch37 %>%
    mutate(entrez = as.character(entrez)) %>%
    select(ensgene, entrez, symbol) %>%
    filter(entrez != "<NA>")

In [16]:
exprs <- de.genes %>%
    inner_join(lookup, by = c("Geneid" = "symbol")) %>% # Convert to entrez
    select(entrez, logFC, P.Value = FDR, symbol = Geneid) %>%
    distinct(entrez, .keep_all = T) %>%
    as.data.frame()

head(exprs, 3)

,entrez,logFC,P.Value,symbol
,<chr>,<dbl>,<dbl>,<chr>
1,8842,-1.918198,3.606216e-275,PROM1
2,389289,-2.123526,3.086909e-222,ANXA2R
3,132864,-1.958025,1.677680e-151,CPEB2


## Generate network using GeneNetworkBuilder, and export nodes and edges

In [24]:
net <- buildNetwork(
    TFbindingTable = as.data.frame(binding.table),
    interactionmap = hs.interactionmap,
    level = 1
)

filtered.net <- filterNetwork(
    rootgene = "4297", # KMT2A entrez ID. Replace as required.
    sifNetwork = net,
    exprsData = exprs,
    mergeBy = "entrez",
    tolerance = 2,
    cutoffPVal = 0.05,
    cutoffLFC = 0.001
)


In [26]:

edges <- filtered.net %>% select(from, to)
nodes <- edges %>% 
    gather(node_type, entrez) %>% 
    distinct(entrez) %>% 
    inner_join(hg38.lookup, by = "entrez") %>% 
    select(entrez, symbol)

    write_tsv(edges, "./SEM_MLL-AF4-Network_edges.txt")
    write_tsv(nodes, "./SEM_MLL-AF4-Network_nodes.txt")